In [16]:
import pandas as pd
from sklearn.manifold import TSNE
import glob
import numpy as np

In [31]:
def projection_coeffs(poly_division, time_interval):
    coeffs = pd.read_csv(f"data/coeffs/{poly_division}_{time_interval}.csv")
    columns_to_keep = ["id_poly", "date"] + [col for col in coeffs.columns if "3" in col]
    feature_names = [col.split("_")[0] for col in coeffs.columns if "3" in col]
    coeffs = coeffs[columns_to_keep]

    coeffs = np.stack(coeffs.groupby("id_poly").apply(lambda x : x.values[:, 2:]))
    
    projections = []
    for i, feat in enumerate(feature_names):
        coeffs_ = coeffs[:, :, i]
        tsne = TSNE(n_components=2, random_state=0)
        proj = tsne.fit_transform(coeffs_)
        # make projection be inside [0.025, 0.975]
        proj = (proj - proj.min(axis=0)) / (proj.max(axis=0) - proj.min(axis=0))
        proj = proj * 0.95 + 0.025
        df = pd.DataFrame(proj, columns=[f"{feat}_x", f"{feat}_y"])
        projections.append(df)
    projections = pd.concat(projections, axis=1)
    projections["id_poly"] = np.arange(len(projections))
    projections.to_csv(f"data/projections/{poly_division}_{time_interval}.csv", index=False)

In [32]:
for time_interval in ["Month", "Day"]:
    for poly_division in ["SpGrid", "SpDistricts", "SpCenterCensus2k", "SpCenterCensus5k"]:
        projection_coeffs(poly_division, time_interval)

/tmp/ipykernel_201416/2544156277.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  coeffs = np.stack(coeffs.groupby("id_poly").apply(lambda x : x.values[:, 2:]))
/tmp/ipykernel_201416/2544156277.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  coeffs = np.stack(coeffs.groupby("id_poly").apply(lambda x : x.values[:, 2:]))
/tmp/ipykernel_201416/2544156277.py:7: DeprecationWarning: DataFrameGroupBy.apply 